In [ ]:
#importações iniciais
import boto3
import pandas as pd
from io import BytesIO
import os
from datetime import datetime
import re
import pyarrow as pa
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import connection
import logging
from botocore.exceptions import ClientError
from decouple import config

In [ ]:
#ver a quantidade de ids que deveria retornar ao final retirando duplicatas.ler o csv que possui os ids que precisava achar no S3 e retirar duplicatas
df_contagem = pd.read_csv(r"C:\Users\documentos-estruturados\list.csv")
df_contagem = df_contagem[['Id']]
contagem = df_contagem[['Id']].value_counts().reset_index()
contagem.columns = ['Id', 'QuantidadeDocumentos']
df_contagem = df_contagem.merge(contagem, on='Id', how='left')
df_contagem = df_contagem.drop_duplicates(subset='Id')


In [ ]:
#conexao banco

conn = connection.MySQLConnection(user='x', password ='x', host ='x', database = " x")
cursor = conn.cursor()

In [ ]:
# Lista para armazenar os resultados das consultas de documentos 1

resultados = []

# Iterar pelos IDs no DataFrame
for _, row in df.iterrows():
    id_procurado = row['Id']
    comando = f"query para localizar os dados procurados = {id_procurado}"
    cursor.execute(comando)
    
    # Executar a consulta SQL e carregar os resultados em um DataFrame
    df_resultado = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    
    # Verifique se há resultados e os adicione à lista
    if not df_resultado.empty:
        resultados.append(df_resultado)



In [ ]:
# Concatenar os resultados em um único DataFrame
if resultados:
    df_documentos1 = pd.concat(resultados, ignore_index=True)
    print(df_documentos1)
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#lista de documentos2
resultados = []

# Iterar pelos IDs no DataFrame
for _, row in df.iterrows():
    id_procurado = row['Id']
    comando = f"query para localizar os dados procurados  = {id_procurado}"
    cursor.execute(comando)
    
    # Executar a consulta SQL e carregar os resultados em um DataFrame
    df_resultado = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    
    # Verifique se há resultados e os adicione à lista
    if not df_resultado.empty:
        resultados.append(df_resultado)

In [ ]:
# Concatenar os resultados em um único DataFrame
if resultados:
    df_documentos2 = pd.concat(resultados, ignore_index=True)
    print(df_documentos2)
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#atribuindo qual era o tipo documento pra cada df gerado
df_documentos1['TipoDocumento'] = 'Documento1'
df_documentos2=['TipoDocumento'] = 'Documento2'

In [ ]:
#variaveis de ambiente
load_dotenv()
config = {
    "bucket": os.getenv('bucket'),
    "path": os.getenv('path')
}

In [ ]:
#login para acessar aws

aws_access_key_id = os.environ['aws_access_key_id']
aws_secret_access_key = os.environ['aws_secret_access_key']


s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [ ]:
# Prefixo do diretório que você deseja encontrar - aqui só retorna os 1000 primeiros
bucket_origem= 'bucket_origem'
diretorio_origem = 'diretorio_origem/'

# Lista os objetos no bucket
response = s3.list_objects_v2(Bucket=bucket_origem)

# Itera sobre os objetos encontrados
for obj in response.get('Contents', []):
    objeto_key = obj['Key']
    
    # Verifica se o objeto é um diretório procurado
    if objeto_key.startswith(diretorio_origem):
        print(f"Diretório encontrado: {objeto_key}")

In [ ]:
#retornar todos (precisa desse def pra retornar todos os objetos dentro de um bucket)
def list_all_objects(bucket_name, prefix):
    s3_client = boto3.client('s3')
    continuation_token = None
    all_files = []
    while True:
        list_kwargs = {
            'Bucket': bucket_name,
            'Prefix': prefix,
        }
        if continuation_token:
            list_kwargs['ContinuationToken'] = continuation_token
        # Fazemos o pedido para listar os objetos
        response = s3_client.list_objects_v2(**list_kwargs)
        # Adicionamos os objetos retornados à nossa lista
        all_files.extend(response.get('Contents', []))
        # Se não houver mais objetos,para o loop
        if not response.get('IsTruncated'):
            break
        # Senão, atualizamos o continuation token para a próxima iteração
        continuation_token = response.get('NextContinuationToken')
    return [file['Key'] for file in all_files]
# Uso
bucket_name = 'bucket_name'
prefix = 'prefix/'
files = list_all_objects(bucket_name, prefix)
print(files)

In [ ]:
#criar a pre signed-url/ expiration em segundos
def create_presigned_url(bucket_name, object_name, expiration=259200):

    # Generate a presigned URL for the S3 object
    s3_client = boto3.client('s3')
    try:
        response = s3_client.generate_presigned_url('get_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name},
                                                    ExpiresIn=expiration)
    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

In [ ]:
#salvar o df_documentos1 com as urls geradas

if resultados:
    
    # Adicionar a coluna 'x' com URLs assinadas
    df_documentos1['Pre_signed'] = df_documentos1.apply(lambda row: create_presigned_url(bucket_name, 'Doc1/' + row['URL']), axis=1)
    
    
    print("DataFrame salvo com URLs assinadas.")
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#salvar o df_documentos2 com as urls geradas

if resultados:
    
    # Adicionar a coluna 'x' com URLs assinadas
    df_documentos2['Pre_signed'] = df_documentos2.apply(lambda row: create_presigned_url(bucket_name, 'Doc2/' + row['URL']), axis=1)
    
    
    print("DataFrame salvo com URLs assinadas.")
else:
    print("Nenhum resultado encontrado para os IDs fornecidos.")

In [ ]:
#concatenar os 3 dfs e colocar em um csv
dataframes = [df_documentos1,df_documentos2]
df_final = pd.concat(dataframes, ignore_index=True)
df_final = df_final[['id','TipoDocumento','Pre_signed']]
df_final.to_csv("Documento.csv",index=False)